In [60]:
# Sub Agent
import glob, os, json
from dotenv import load_dotenv

load_dotenv()

# Step convert human message to sql statement
import pandas as pd
from typing import Literal
from services.chatgpt_connection import ChatGPTConnection
from services.sqlite_connection import SQLiteConnection
from services.postgresql_connection import PostgreSQLConnection


DB_MAPPER: dict[str, SQLiteConnection, PostgreSQLConnection] = {
    "chinook-db-agent": SQLiteConnection(
        os.environ.get("SQLITE_DB_PATH", "database/chinook.db"),
    ),
    "northwind-db-agent": PostgreSQLConnection(
        db_name=os.environ.get("POSTGRES_DB_NAME", "northwind"),
        user=os.environ.get("POSTGRES_USER", "postgres"),
        password=os.environ.get("POSTGRES_PASSWORD", "postgres"),
        host=os.environ.get("POSTGRES_HOST", "localhost"),
        port=5432,
    ),
}

In [69]:
sub_agents: list[dict] = []
for json_file in glob.glob("agents/*.json"):
    sub_agent_config: dict = json.loads(open(json_file, "r", encoding="utf-8").read())
    # set prompt from txt file
    if sub_agent_config.get("prompt_file_path") and sub_agent_config.get("sql_schema_path"):
        txt_structure_sql = open(sub_agent_config["sql_schema_path"], "r", encoding="utf-8").read()
        sub_agent_config["prompt"] = open(
            str(sub_agent_config.get("prompt_file_path")),
            "r",
            encoding="utf-8",
        ).read()

        sub_agent_config["prompt"] = str(sub_agent_config["prompt"]).format(db_structure_txt=txt_structure_sql)
        sub_agents.append(sub_agent_config)
sub_agents

[{'agent_name': 'northwind-db-agent',
  'description': 'Agent who know everything about categories,customer_customer_demo,customer_demographics,customers,employee_territories,employees,order_details,orders,products,region,shippers,suppliers,territories,us_states',
  'db_type': 'postgresql',
  'prompt_file_path': 'prompts/northwind-db-agent.txt',
  'sql_schema_path': 'database/northwind-structure.sql',
  'prompt': 'คุณคือผู้ช่วยตอบคำถามของ User โดยคุณจะรู้ข้อมูลของ database `northwind` (Postgresql)\n\nDATABASE SCHEMA:\n```-- -------------------------------------------------------------\n-- TablePlus 6.1.2(568)\n--\n-- https://tableplus.com/\n--\n-- Database: northwind\n-- Generation Time: 2567-08-27 11:56:28.4640\n-- -------------------------------------------------------------\n\n\n-- This script only contains the table creation statements and does not fully represent the table in the database. Do not use it as a backup.\n\n-- Table Definition\nCREATE TABLE "public"."categories" (\n   

In [70]:
def convert_query_result_to_human_message(user_question: str, sql_statement: str, query_records: list, excel_path: str):
    sql_to_human_client_agent: ChatGPTConnection = ChatGPTConnection(
        api_key=os.environ.get("AZURE_OPENAI_KEY"),
        api_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
        api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
        model="gpt-4o-mini",
        system_role=open("prompts/convert_query_records_to_human.txt", "r", encoding="utf-8").read(),
        output_structure={
            "human_message": "text",
        },
    )

    user_prompt = """
    User Question : {user_question}
    SQL Statment : {sql_statement}
    Query Result : {query_records}
    Excel Path : {excel_path}
    You : """

    # if len(query_records) > 10:
    #     query_records = str(query_records[0:10]) + ".. {} records left.".format(len(query_records) - 10)

    final_prompt = user_prompt.format(
        user_question=user_question,
        sql_statement=sql_statement,
        query_records=str(query_records),
        excel_path=excel_path,
    )
    # print(final_prompt)

    result_human = sql_to_human_client_agent.query_chat(prompt=final_prompt)
    return result_human["human_message"]

In [117]:
def chat_with_agent(agent_name=sub_agents[0]["agent_name"], input_message: str = ""):

    use_agents: list[dict] = [s for s in sub_agents if s["agent_name"] == agent_name]
    use_agent: dict = None
    if use_agents:
        use_agent = use_agents[0]

    client_agent = ChatGPTConnection(
        api_key=os.environ.get("AZURE_OPENAI_KEY"),
        api_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
        api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
        model="gpt-4o-mini",
        system_role=use_agent.get("prompt"),
        output_structure={
            "sql_statement": "text (only query statement without comments)",
        },
    )
    generate_sql_result = client_agent.query_chat(input_message)

    db_executor = DB_MAPPER[agent_name]
    db_executor.connect()
    # print("## Step 1 Generate Statement >>", generate_sql_result["sql_statement"])

    query_records = db_executor.fetch_all(query=generate_sql_result["sql_statement"])
    # query_records_pd = pd.DataFrame(query_records)
    # excel_path = "assets/{}.xlsx".format(input_message)
    # query_records_pd.to_excel(excel_path)

    # print("## Step 2 Query result >>", query_records)

    # print("## Step 3 Human Pretty Message >>")
    return_human_message = convert_query_result_to_human_message(
        user_question=input_message,
        sql_statement=generate_sql_result["sql_statement"],
        query_records=query_records,
        excel_path=None,
    )
    return return_human_message

In [118]:
# test chinook db
chat_with_agent(agent_name="chinook-db-agent", input_message="ขอดูรายชื่อลูกค้าทั้งหมดหน่อย")

Connected to the database at database/chinook.db


'นี่คือรายชื่อลูกค้าทั้งหมด:\n\n1. Luís Gonçalves - Embraer - Empresa Brasileira de Aeronáutica S.A., São José dos Campos, SP, Brazil, โทร: +55 (12) 3923-5555, อีเมล: luisg@embraer.com.br\n2. Leonie Köhler - Theodor-Heuss-Straße 34, Stuttgart, Germany, โทร: +49 0711 2842222, อีเมล: leonekohler@surfeu.de\n3. François Tremblay - 1498 rue Bélanger, Montréal, QC, Canada, โทร: +1 (514) 721-4711, อีเมล: ftremblay@gmail.com\n4. Bjørn Hansen - Ullevålsveien 14, Oslo, Norway, โทร: +47 22 44 22 22, อีเมล: bjorn.hansen@yahoo.no\n5. František Wichterlová - JetBrains s.r.o., Klanova 9/506, Prague, Czech Republic, โทร: +420 2 4172 5555, อีเมล: frantisekw@jetbrains.com\n6. Helena Holý - Rilská 3174/6, Prague, Czech Republic, โทร: +420 2 4177 0449, อีเมล: hholy@gmail.com\n7. Astrid Gruber - Rotenturmstraße 4, 1010 Innere Stadt, Vienne, Austria, โทร: +43 01 5134505, อีเมล: astrid.gruber@apple.at\n8. Daan Peeters - Grétrystraat 63, Brussels, Belgium, โทร: +32 02 219 03 03, อีเมล: daan_peeters@apple.be\n

In [119]:
# test chinook db
chat_with_agent(agent_name="northwind-db-agent", input_message="ขอดูรายชื่อประเทศทั้งหมดหน่อย")

Connected to the PostgreSQL database northwind at localhost:5432


'รายชื่อประเทศทั้งหมด ได้แก่ อาร์เจนตินา, สเปน, สวิตเซอร์แลนด์, อิตาลี, เวเนซุเอลา, เบลเยียม, นอร์เวย์, สวีเดน, สหรัฐอเมริกา, ฝรั่งเศส, เม็กซิโก, บราซิล, ออสเตรีย, โปแลนด์, สหราชอาณาจักร, ไอร์แลนด์, เยอรมนี, เดนมาร์ก, แคนาดา, ฟินแลนด์, และโปรตุเกส.'

In [120]:
# Build main agent
sub_agents

core_prompt = open("prompts/core.txt", "r").read()
interact_with_agents = ""
for idx, s_agent in enumerate(sub_agents):
    interact_with_agents += "[{idx}] Agent Name : `{agent_name}`, JobDescription : `{description}`\n".format(
        idx=idx,
        agent_name=s_agent["agent_name"],
        description=s_agent["description"],
    )
print(core_prompt.format(interact_with_agents=interact_with_agents))

user_question = "อัลบัมน์อะไรขายดีที่สุดและลูกค้าประเทศอะไรสั่งซื้อเยอะที่สุดใน chinook, แล้วประเทศดังกล่าวมีลูกค้ากี่รายใครบ้างใน northwind?"
# === Step 1
prompt_list_query_message_to_agent_task = core_prompt.format(interact_with_agents=interact_with_agents)
create_list_todo_client_agent = ChatGPTConnection(
    api_key=os.environ.get("AZURE_OPENAI_KEY"),
    api_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
    model="gpt-4o-mini",
    system_role=prompt_list_query_message_to_agent_task,
    output_structure={
        "todo_tasks": "list[dict] \{'agent_name':'str', 'question_to_agent':'str'\}",
    },
)
todo_tasks = create_list_todo_client_agent.query_chat(user_question)
todo_tasks["todo_tasks"]

คุณคือผู้ช่วยตอบคำถามของ User โดยคุณจะรู้จักกับ Agent ดังต่อไปนี้

[0] Agent Name : `northwind-db-agent`, JobDescription : `Agent who know everything about categories,customer_customer_demo,customer_demographics,customers,employee_territories,employees,order_details,orders,products,region,shippers,suppliers,territories,us_states`
[1] Agent Name : `chinook-db-agent`, JobDescription : `Agent who know everything about albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracks`



[{'agent_name': 'chinook-db-agent',
  'question_to_agent': 'อัลบัมน์อะไรขายดีที่สุด?'},
 {'agent_name': 'chinook-db-agent',
  'question_to_agent': 'ลูกค้าประเทศอะไรสั่งซื้อเยอะที่สุด?'},
 {'agent_name': 'northwind-db-agent',
  'question_to_agent': 'ประเทศที่มีลูกค้าสั่งซื้อเยอะที่สุดมีลูกค้ากี่รายและใครบ้าง?'}]

In [123]:
prev_row = None
query_result = None
for idx, item_task in enumerate(todo_tasks["todo_tasks"]):
    print(idx, "----")
    user_q = item_task["question_to_agent"]
    agent_name = item_task["agent_name"]
    if prev_row is None:
        query_result = chat_with_agent(agent_name=agent_name, input_message=user_q)
        prev_row = "{agent_name}: {query_result}".format(agent_name=agent_name, query_result=query_result)

    else:
        query_result = chat_with_agent(
            agent_name=agent_name,
            input_message="{prev_row}\n{user_q}".format(
                prev_row=prev_row,
                user_q=user_q,
            ),
        )
        prev_row = "{prev_row}\n{agent_name}: {query_result}".format(prev_row=prev_row, agent_name=agent_name, query_result=query_result)

    print(prev_row)

print(query_result)

0 ----
Connected to the database at database/chinook.db
chinook-db-agent: อัลบัมน์ที่ขายดีที่สุดคือ 'Minha Historia' โดยมียอดขายทั้งหมด 27 ชุด.
1 ----
Connected to the database at database/chinook.db
chinook-db-agent: อัลบัมน์ที่ขายดีที่สุดคือ 'Minha Historia' โดยมียอดขายทั้งหมด 27 ชุด.
chinook-db-agent: ลูกค้าที่สั่งซื้อเยอะที่สุดมาจากประเทศสหรัฐอเมริกา โดยมียอดสั่งซื้อทั้งหมด 91 รายการ.
2 ----
Connected to the PostgreSQL database northwind at localhost:5432
chinook-db-agent: อัลบัมน์ที่ขายดีที่สุดคือ 'Minha Historia' โดยมียอดขายทั้งหมด 27 ชุด.
chinook-db-agent: ลูกค้าที่สั่งซื้อเยอะที่สุดมาจากประเทศสหรัฐอเมริกา โดยมียอดสั่งซื้อทั้งหมด 91 รายการ.
northwind-db-agent: ประเทศที่มีลูกค้าสั่งซื้อเยอะที่สุดคือ สหรัฐอเมริกา โดยมีลูกค้าทั้งหมด 13 ราย.
ประเทศที่มีลูกค้าสั่งซื้อเยอะที่สุดคือ สหรัฐอเมริกา โดยมีลูกค้าทั้งหมด 13 ราย.


In [114]:
query_result

'ประเทศที่มีลูกค้าสั่งซื้อเยอะที่สุดคือประเทศที่มีลูกค้าทั้งหมด 13 ราย.'